<style>
    body, * {
        direction: rtl !important;
        text-align: right !important;
    }
</style>

# 🔥 מדריך מקיף ל-PySpark - טרנספורמציות ופעולות

מחברת זו מכסה את הנושאים הבאים:
1. **map()** - טרנספורמציה יסודית - [map](https://sparkbyexamples.com/pyspark/pyspark-map-transformation/)
2. **flatMap()** - טרנספורמציה משטחת - [flatMap](https://sparkbyexamples.com/pyspark/pyspark-flatmap-transformation/)
3. **foreach()** - איטרציה על אלמנטים - [foreach](https://sparkbyexamples.com/pyspark/pyspark-foreach-usage-with-examples/)
4. **sampling()** - דגימה אקראית - [sampling](https://sparkbyexamples.com/pyspark/pyspark-sampling-example/)
5. **fillna() & fill()** - החלפת ערכי NULL - [fillna() & fill()](https://sparkbyexamples.com/pyspark/pyspark-fillna-fill-replace-null-values/)
---

## הגדרות ראשוניות

ראשית, נייבא את הספריות הנדרשות וניצור SparkSession:

In [ ]:
from pyspark.sql import SparkSession

# יצירת SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("SparkByExamples.com") \
    .getOrCreate()

---

# 1. map() - טרנספורמציית מיפוי 🗺️

## מהו map()?

הפונקציה `map()` ב-PySpark היא טרנספורמציה המשמשת להחלת פונקציה/למבדה על כל אלמנט ב-RDD (Resilient Distributed Dataset) ומחזירה RDD חדש המכיל את התוצאות.

### 🔑 נקודות מפתח:

- DataFrame אינו תומך בטרנספורמציה map(), לכן יש להמיר אותו ל-RDD תחילה
- אם יש לך אתחול כבד, השתמש ב-mapPartitions() במקום map(), מכיוון שאתחול כבד מתבצע רק פעם אחת לכל מחיצה במקום לכל רשומה

## תחביר (Syntax)

In [ ]:
# תחביר
# map(f, preservesPartitioning=False)

## דוגמה 1: יצירת RDD מרשימה

In [ ]:
# יצירת נתונים לדוגמה
data = ["Project", "Gutenberg's", "Alice's", "Adventures",
        "in", "Wonderland", "Project", "Gutenberg's", "Adventures",
        "in", "Wonderland", "Project", "Gutenberg's"]

# יצירת RDD
rdd = spark.sparkContext.parallelize(data)

# הדפסת האלמנטים
print(rdd.collect())

## דוגמה 2: שימוש ב-map() עם RDD

כאן נחיל טרנספורמציה map() על RDD. בדוגמה הבאה, תחילה היא מפצלת כל רשומה לפי רווח ב-RDD ולבסוף משטחת אותו. התוצאה של ה-RDD מורכבת ממילה אחת בכל רשומה.

In [ ]:
# החלת map() - יצירת tuples עם הערך 1
rdd2 = rdd.map(lambda x: (x, 1))

# הדפסת התוצאות
print(rdd2.collect())


## דוגמה 3: שימוש ב-map() עם DataFrame

למרבה הצער, PySpark DataFrame אינו תומך בטרנספורמציה map() להחלת הפונקציה למבדה, כאשר רוצים להחיל טרנספורמציה מותאמת אישית, יש להמיר את ה-DataFrame ל-RDD ולהחיל את הטרנספורמציה map().

In [ ]:
# יצירת DataFrame לדוגמה
data = [('James', 'Smith', 'M', 30),
        ('Anna', 'Rose', 'F', 41),
        ('Robert', 'Williams', 'M', 62)]

columns = ["firstname", "lastname", "gender", "salary"]
df = spark.createDataFrame(data=data, schema=columns)
df.show()

### שימוש ב-map() - התייחסות לעמודות לפי אינדקס

In [ ]:
# המרה ל-RDD והחלת map
rdd2 = df.rdd.map(lambda x: (x[0] + "," + x[1], x[2], x[3] * 2))

df2 = rdd2.toDF(["name", "gender", "new_salary"])
df2.show()

### שימוש ב-map() - התייחסות לעמודות לפי שם

In [ ]:
# התייחסות לעמודות לפי שם
rdd2 = df.rdd.map(lambda x: 
    (x["firstname"] + "," + x["lastname"], x["gender"], x["salary"] * 2)
)

df2 = rdd2.toDF(["name", "gender", "new_salary"])
df2.show()

### שימוש בפונקציה מותאמת אישית עם map()

In [ ]:
# הגדרת פונקציה מותאמת
def func1(x):
    firstName = x.firstname
    lastName = x.lastname
    name = firstName + "," + lastName
    gender = x.gender.lower()
    salary = x.salary * 2
    return (name, gender, salary)

# החלת הפונקציה
rdd2 = df.rdd.map(lambda x: func1(x))
df2 = rdd2.toDF(["name", "gender", "salary"])
df2.show()

---

# 2. flatMap() - טרנספורמציית מיפוי שטוח 📋

## מהו flatMap()?

ה-`flatMap()` של PySpark היא טרנספורמציה שמשטחת את ה-RDD/DataFrame (מערך/מפת עמודות DataFrame) לאחר החלת הפונקציה על כל אלמנט ומחזירה RDD/DataFrame חדש של PySpark.

## תחביר

In [ ]:
# תחביר
# flatMap(f, preservesPartitioning=False)

## דוגמה: יצירת RDD והחלת flatMap()

In [ ]:
data = ["Project Gutenberg's",
        "Alice's Adventures in Wonderland",
        "Project Gutenberg's",
        "Adventures in Wonderland",
        "Project Gutenberg's"]

rdd = spark.sparkContext.parallelize(data)
print(rdd.collect())

### החלת flatMap() עם פיצול

In [ ]:
# פיצול כל רשומה לפי רווח
rdd2 = rdd.flatMap(lambda x: x.split(" "))
print(rdd2.collect())

## שימוש ב-flatMap() עם DataFrame

למרבה הצער, PySpark DataFrame אינו תומך בטרנספורמציה flatMap(), אולם יש לנו פונקציית SQL `explode()` שמשמשת לשטח את העמודה.

In [ ]:
from pyspark.sql.functions import explode

arrayData = [
    ('James', ['Java', 'Scala'], {'hair': 'black', 'eye': 'brown'}),
    ('Michael', ['Spark', 'Java', None], {'hair': 'brown', 'eye': None}),
    ('Robert', ['CSharp', ''], {'hair': 'red', 'eye': ''}),
    ('Washington', None, None),
    ('Jefferson', ['1', '2'], {})
]

df = spark.createDataFrame(data=arrayData, schema=['name', 'knownLanguages', 'properties'])
df2 = df.select(df.name, explode(df.knownLanguages))
df2.printSchema()
df2.show()

---

# 3. foreach() - לולאה על אלמנטים 🔄

## מהו foreach()?

ה-`foreach()` של PySpark היא פעולת action הזמינה ב-RDD, DataFrame לעבור/לולאה על כל אלמנט ב-DataFrame. הוא דומה ל-for עם קונספטים מתקדמים. זה שונה מפעולות אחרות מכיוון שפונקציה foreach() אינה מחזירה ערך, במקום זאת היא מבצעת את פונקציית הקלט על כל אלמנט של RDD, DataFrame.

## תחביר

In [ ]:
# תחביר
# DataFrame.foreach(f)

## דוגמה 1: foreach() עם DataFrame

In [ ]:
# יצירת DataFrame לדוגמה
columns = ["Seqno", "Name"]
data = [("1", "john jones"),
        ("2", "tracey smith"),
        ("3", "amy sanders")]

df = spark.createDataFrame(data=data, schema=columns)
df.show()

# דוגמה foreach()
def f(df):
    print(df.Seqno)

df.foreach(f)

### שימוש ב-foreach() עם accumulator

In [ ]:
# דוגמה foreach() עם accumulator
accum = spark.sparkContext.accumulator(0)
df.foreach(lambda x: accum.add(int(x.Seqno)))
print(accum.value)  # נגיש על ידי הדרייבר

## דוגמה 2: foreach() עם RDD

In [ ]:
# דוגמה foreach() עם RDD
accum = spark.sparkContext.accumulator(0)
rdd = spark.sparkContext.parallelize([1, 2, 3, 4, 5])
rdd.foreach(lambda x: accum.add(x))
print(accum.value)

---

# 4. sample() - דגימה אקראית 🎲

## מהו sample()?

PySpark מספק פונקציות לדגימה אקראית:
- `pyspark.sql.DataFrame.sample()`
- `pyspark.sql.DataFrame.sampleBy()`
- `RDD.sample()`
- `RDD.takeSample()`

## תחביר

In [ ]:
# תחביר
# sample(withReplacement, fraction, seed=None)

## 1.1 שימוש ב-fraction לקבלת דגימה אקראית

In [ ]:
df = spark.range(100)
print(df.sample(0.06).collect())

## 1.2 שימוש ב-seed לשחזור אותן דגימות

In [ ]:
print(df.sample(0.1, 123).collect())
print(df.sample(0.1, 123).collect())
print(df.sample(0.1, 456).collect())

## 1.3 דגימה עם withReplacement

In [ ]:
print(df.sample(True, 0.3, 123).collect())  # עם כפילויות
print(df.sample(0.3, 123).collect())  # ללא כפילויות

## 1.4 דגימה מרובדת (Stratified Sampling)

In [ ]:
df2 = df.select((df.id % 3).alias("key"))
print(df2.sampleBy("key", {0: 0.1, 1: 0.2}, 0).collect())

## 2. שימוש ב-sample() עם RDD

In [ ]:
rdd = spark.sparkContext.range(0, 100)
print(rdd.sample(False, 0.1, 0).collect())
print(rdd.sample(True, 0.3, 123).collect())

## שימוש ב-takeSample() עם RDD

In [ ]:
print(rdd.takeSample(False, 10, 0))
print(rdd.takeSample(True, 30, 123))

---

# 5. fillna() & fill() - מילוי ערכים חסרים 🔧

## מה זה fillna() ו-fill()?

ב-PySpark, `fillna()` מהמחלקה DataFrame או `fill()` מ-DataFrameNaFunctions משמשים להחלפת ערכי NULL/None.

## תחביר

In [ ]:
# fillna(value, subset=None)
# fill(value, subset=None)

## יצירת DataFrame לדוגמה עם ערכים חסרים

In [ ]:
from pyspark.sql import Row

data = [
    Row(id=1, zipcode=704, type="STANDARD", city=None, state="PR", population=None),
    Row(id=2, zipcode=704, type=None, city="PASEO COSTA DEL SUR", state="PR", population=None),
    Row(id=3, zipcode=709, type=None, city="BDA SAN LUIS", state="PR", population=13700),
    Row(id=4, zipcode=76166, type="UNIQUE", city="CINGULAR WIRELESS", state="TX", population=84000),
    Row(id=5, zipcode=76177, type="STANDARD", city=None, state="TX", population=None)
]

df = spark.createDataFrame(data)
df.printSchema()
df.show(truncate=False)

## 1. החלפת ערכי NULL עם אפס (0)

In [ ]:
# החלף 0 עבור null עבור כל עמודות המספרים השלמים
df.na.fill(value=0).show()

# החלף 0 עבור null רק בעמודת population
df.na.fill(value=0, subset=["population"]).show()

## 2. החלפת NULL עם מחרוזת ריקה

In [ ]:
df.na.fill("").show()

## 3. החלפת nulls בעמודות ספציפיות

In [ ]:
df.na.fill("unknown", ["city"]) \
    .na.fill("", ["type"]).show()

## 4. שימוש ב-dictionary למילוי ערכים שונים

In [ ]:
df.na.fill({"city": "unknown", "type": ""}).show()

---

## 🎓 סיכום

במדריך זה למדנו:

1. **map()** - טרנספורמציה להחלת פונקציה על כל אלמנט ב-RDD
2. **flatMap()** - טרנספורמציה ששטחה את התוצאות לרשימה אחת
3. **foreach()** - action לביצוע פעולה על כל אלמנט (ללא החזרת ערך)
4. **sample()** - קבלת דגימה אקראית מהנתונים
5. **fillna()** - מילוי ערכים חסרים (NULL/None)

---

**Happy Learning! 🚀**

---

## 📚 מקורות

- [Apache Spark Documentation](https://spark.apache.org/docs/latest/api/python/)
- [SparkByExamples.com](https://sparkbyexamples.com/)